# Modelo 1 – Clasificadro de CV

In [ ]:

from tensorflow.keras.utils import to_categorical
from tensorflow import keras



Importación del df

In [ ]:
import pandas as pd

df = pd.read_csv('/Users/alexmiliandieguez/code/TechCareer/data/model1_cv_role/3.processed/v1_english/cv_labeled_final.csv')
df.head()

,cv_id,cv_text,role_label_final,role_raw,source_dataset,resume_len
0,1,Python Developer Python Developer Python Devel...,python_developer,Python_Developer,dataset1_avishek,3467
1,2,R&D Engineer R&D Engineer R&D Engineer - Nokia...,python_developer,Python_Developer,dataset1_avishek,2812
2,3,Sr. Full Stack Developer Sr. Full Stack Develo...,python_developer,Python_Developer,dataset1_avishek,16606
3,4,Sr. Full Stack Python Developer Sr. Full Stack...,python_developer,Python_Developer,dataset1_avishek,9253
4,5,Sr. Python Developer Sr. Python Developer Sr. ...,python_developer,Python_Developer,dataset1_avishek,15659


Quitamos columnas que no necesitamos

In [26]:
df = df.loc[:, ['cv_text', 'role_label_final']]

df.rename(columns={'cv_text': 'cv', 'role_label_final': 'role'}, inplace=True)
df.head()


,cv,role
0,Python Developer Python Developer Python Devel...,python_developer
1,R&D Engineer R&D Engineer R&D Engineer - Nokia...,python_developer
2,Sr. Full Stack Developer Sr. Full Stack Develo...,python_developer
3,Sr. Full Stack Python Developer Sr. Full Stack...,python_developer
4,Sr. Python Developer Sr. Python Developer Sr. ...,python_developer


Separación de etiquetas + train test

In [ ]:
from sklearn.model_selection import train_test_split

X_texts = df['cv'].values 
y_labels = df['role'].values 

X_train_texts, X_test_texts, y_train_labels, y_test_labels = train_test_split(
    X_texts, y_labels, test_size=0.2, random_state=42)


Vectorizar CV (máx 5000 carácteres distintos)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(X_train_texts).toarray()
X_test = vectorizer.transform(X_test_texts).toarray()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_train = le.fit_transform(y_train_labels)
y_test = le.transform(y_test_labels)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_train.shape[1]


In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 1024)           │     5,121,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 16)             │         8,208 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,654,032 (21.57 MB)

 Trainable params: 5,654,032 (21.57 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=20,
                    validation_data=(X_test, y_test),
                    verbose=2)


Epoch 1/20
207/207 - 24s - 114ms/step - accuracy: 0.1416 - loss: 2.3643 - val_accuracy: 0.1627 - val_loss: 2.3321
Epoch 2/20
207/207 - 20s - 96ms/step - accuracy: 0.1449 - loss: 2.3460 - val_accuracy: 0.1627 - val_loss: 2.3265
Epoch 3/20
207/207 - 20s - 97ms/step - accuracy: 0.1451 - loss: 2.3389 - val_accuracy: 0.1434 - val_loss: 2.3281
Epoch 4/20
207/207 - 20s - 99ms/step - accuracy: 0.1448 - loss: 2.3392 - val_accuracy: 0.1434 - val_loss: 2.3436
Epoch 5/20
207/207 - 21s - 100ms/step - accuracy: 0.1508 - loss: 2.3370 - val_accuracy: 0.1627 - val_loss: 2.3263
Epoch 6/20
207/207 - 20s - 95ms/step - accuracy: 0.1478 - loss: 2.3354 - val_accuracy: 0.1434 - val_loss: 2.3277
Epoch 7/20
207/207 - 21s - 100ms/step - accuracy: 0.1484 - loss: 2.3320 - val_accuracy: 0.1627 - val_loss: 2.3265
Epoch 8/20
207/207 - 17s - 83ms/step - accuracy: 0.1457 - loss: 2.3317 - val_accuracy: 0.1155 - val_loss: 2.3718
Epoch 9/20
207/207 - 17s - 84ms/step - accuracy: 0.1505 - loss: 2.3343 - val_accuracy: 0.1434

KeyboardInterrupt: 